In [ ]:
import cv2
from cvzone.HandTrackingModule import HandDetector
from cvzone.ClassificationModule import Classifier
import numpy as np
import math
import time
import speech_recognition as sr


In [ ]:
cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=1)
classifier = Classifier(r"C:\Users\salam_jmvmbso\Downloads\Model\keras_model.h5", r"C:\Users\salam_jmvmbso\Downloads\Model\labels.txt")


In [ ]:
offset = 20
imgSize = 300

folder =r"C:\Users\salam_jmvmbso\Downloads\data\data"
counter = 0
labels = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]
recognizer = sr.Recognizer()

In [ ]:
while True:
    success, img = cap.read()
    imgOutput = img.copy()
    hands, img = detector.findHands(img)

    if hands:
        hand = hands[0]
        x, y, w, h = hand["bbox"]

        imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
        imgCrop = img[y-offset:y + h+offset, x-offset:x + w+offset]

        imgCropShape = imgCrop.shape

        aspectratio = h / w

        if aspectratio > 1:
            k = imgSize / h
            wCal = math.ceil(k * w)
            imgResize = cv2.resize(imgCrop, (wCal, imgSize))
            imgResizeShape = imgResize.shape
            wGap = math.ceil((imgSize - wCal)/2)
            imgWhite[:, wGap:wGap+wCal] = imgResize
            prediction, index = classifier.getPrediction(imgWhite, draw=False)
            print(prediction, index)



        elif aspectratio < 1:
            k = imgSize / w
            hCal = math.ceil(k * h)
            imgResize = cv2.resize(imgCrop, (imgSize, hCal))
            imgResizeShape = imgResize.shape
            hGap = math.ceil((imgSize -hCal) / 2)
            imgWhite[hGap:hGap+hCal, :] = imgResize
            prediction, index = classifier.getPrediction(imgWhite, draw=False)
            print(prediction,  index)

        with sr.Microphone() as source:
            print("Say a command:")
            audio = recognizer.listen(source)

        try:
            # Recognize the voice command
            voice_command = recognizer.recognize_google(audio).lower()
            print("You said:", voice_command)

            # Check if the recognized command is in your list of labels
            if voice_command in labels:
                index = labels.index(voice_command)
                cv2.rectangle(imgOutput, (x - offset, y - offset-50), (x - offset+90, y + offset-50+50), (255, 0, 255), cv2.FILLED)
                cv2.putText(imgOutput, labels[index], (x, y-20), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 2)
                cv2.rectangle(imgOutput, (x-offset, y-offset), (x + w+offset, y + h+offset), (255, 0, 255), 4)


        except sr.UnknownValueError:
            print("Could not understand audio")
        except sr.RequestError as e:
            print(f"Could not request results from Google Speech Recognition service; {e}")

        cv2.imshow("ImageCrop", imgCrop)
        cv2.imshow("ImageWhite", imgWhite)


In [ ]:
cv2.imshow("Image", imgOutput)
cv2.waitKey(1)